In [ ]:
!pip install transliterate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.8 MB/s eta 0:00:00


In [ ]:
import random
import datetime
import os
import argparse
from transliterate import translit
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import col, when, count

In [ ]:
# Список из 25 имен
prepared_list_names = [
    "Алексей", "Ольга", "Дмитрий", "Мария", "Иван",
    "Анна", "Сергей", "Елена", "Андрей", "Наталья",
    "Владимир", "Татьяна", "Михаил", "Светлана", "Кирилл",
    "Юлия", "Александр", "Екатерина", "Николай", "Ирина",
    "Василий", "Евгений", "Людмила", "Павел", "Роман"
]

# Список из 40 городов
prepared_list_cities = [
    "Москва", "Санкт-Петербург", "Новосибирск", "Екатеринбург", "Казань",
    "Нижний Новгород", "Челябинск", "Самара", "Омск", "Ростов-на-Дону",
    "Уфа", "Красноярск", "Воронеж", "Пермь", "Волгоград",
    "Краснодар", "Тольятти", "Ижевск", "Ульяновск", "Барнаул",
    "Тюмень", "Иркутск", "Саратов", "Хабаровск", "Ярославль",
    "Владивосток", "Махачкала", "Томск", "Оренбург", "Кемерово",
    "Новокузнецк", "Рязань", "Астрахань", "Пенза", "Липецк",
    "Тула", "Киров", "Брянск", "Чебоксары", "Калининград"
]


In [42]:
current_date = datetime.date.today()

In [ ]:
# ф-ция генерации даты рождения
def generate_birth_date(age: int) -> datetime.date:
    # Расчет даты рождения для случайного возраста
    random_date_of_birth = current_date - datetime.timedelta(days=age * 365)

    return random_date_of_birth

In [ ]:
# ф-ция генерации даты регистрации.
# Ограничения:
# регистрация возможна после достижения человеком пяти лет и до текущей даты.
def generate_registration_date(birth_date: datetime.date) -> datetime.date:
    # toordinal() переводит дату в дни, а fromordinal() обратно в дату
    start_date = birth_date.toordinal() + 5*365
    end_date = current_date.toordinal()
    random_registr_date = datetime.date \
        .fromordinal(random.randint(start_date, end_date))

    return random_registr_date

In [ ]:
# ф-ции выбора случайного имени и города из отфильтрованного списка
def random_name() -> str:
  return random.choice([name for name in prepared_list_names if len(name) >= 5])

def random_city() -> str:
  return random.choice([city for city in prepared_list_cities if len(city) >= 7])

In [ ]:
# ф-ция переименовывает .csv файлы в формат "текущий год-месяц-день-dev.csv" и
# удаляет лишние файлы
def rename_csv(output_path, date: datetime.date = current_date) -> str:
    files_list = os.listdir(output_path)
    new_filename = output_path + f'/{date}-dev.csv'

    for file in files_list:
        if file.startswith('part-') and file.endswith('.csv'):
            old_name = output_path + '/' + file
            os.rename(old_name, new_filename)
        elif not file.endswith('dev.csv'):
            os.remove(os.path.join(output_path, file))

    return new_filename

In [ ]:
# ф-ция генерации данных
def generate_data(num_rows: int) -> list[tuple]:
    data = []

    # Рассчитываем количество NULL значений (не более 5% в каждом столбце)
    number_null_in_table = int(num_rows * 0.05)

    # Для каждого столбца создаем свой счетчик
    null_counts = {
        'name': 0,
        'email': 0,
        'city': 0,
        'age': 0,
        'salary': 0,
        'registration_date': 0
    }

    # Генерация данных
    for i in range(0, num_rows):
        name = random_name()
        city = random_city()
        age = random.randint(18, 95) # Генерация случайного возраста от 18 до 95 лет
        birth_date = generate_birth_date(age)
        registration_date = generate_registration_date(birth_date)
        #во время преобразования символов из ru в en translit заменяет "ь" на "'' (Ольга -> Ol'ga)
        # с помощью .replace("'", "") убираем "'" (Ольга -> Olga)
        email = translit(name.lower(), 'ru', reversed=True).replace("'", "")\
        + str(birth_date.year) + '_' + str(birth_date.day) + '@'\
        + random.choice(['ru', 'com'])
        salary = random.choice(range(100, 500)) * 10**3

        # промежуточная запись в словарь
        dict_data ={
            'name':name,
            'city':city,
            'age':age,
            'registration_date':registration_date,
            'email':email,
            'salary':salary
        }

        # Проверяем для каждого ключа (столбца) условие для вставки NULL значений
        for k, v in dict_data.items():
            #1. счетчик для столбца <= допустимому кол-ву NULL значений по столбцу
            #2. вероятность вставки NULL (если random.random() возвращает число меньше 0.05)
            if null_counts[k] <= number_null_in_table and random.random() < 0.05:
                dict_data[k] = None
                null_counts[k] += 1  # Увеличиваем счетчик для этого столбца


        # Создаем кортеж данных и добавляем его в итоговый список
        row_data = (i + 1, dict_data['name'], dict_data['email'], dict_data['city'],\
                    dict_data['age'], dict_data['salary'], dict_data['registration_date'])

        data.append(row_data)

    return data



In [ ]:
# ф-ция создания 'DataFrame'
def create_df(spark: SparkSession, schema: StructType, data: list[tuple]) -> 'DataFrame':

    df = spark.createDataFrame(data=data, schema=schema)

    return df

In [ ]:
# ф-ция проверки условия наличия NULL значений в данных
def check_null_value(df: 'DataFrame', count_data: int, columns_list: list) -> None:
    # ф-ция проверки условия на превышение NULL значений по столбцам (не прывышать 5% в каждом из столбцов)
    def check_null_percentage(column_name: str, cnt_null_colum: int, count_data: int) -> None:
        null_percentage = (cnt_null_colum / count_data) * 100
        if null_percentage > 5 and null_percentage <= 6:
            print(f'количество NULL в столбце "{column_name}" незначительно превышает 5% и равна {null_percentage:.2f}%')
        elif null_percentage > 6:
            print(f'количество NULL в столбце "{column_name}" ЗНАЧИТЕЛЬНО превышает 5% и равна {null_percentage:.2f}%')
        else:
            print(f'количество NULL в столбце "{column_name}" не превышает 5% и равна {null_percentage:.2f}%')

    for colum in columns_list:
        # количество NULL значений по столбцам
        cnt_null_colum = df.filter(col(colum).isNull()).count()
        check_null_percentage(colum, cnt_null_colum, count_data)


In [ ]:
# ф-ция записи данных в .csv
def data_in_csv(df: 'DataFrame', report_date: datetime.date, output_path: str, count_data: int) -> None:

    df.coalesce(1).write.csv(output_path, header=True, mode="append")
    filename = rename_csv(output_path, report_date)

    print(f'Было сгенерировано {count_data} строк данных, \
которые были записаны в файл \'{filename}\' за дату {report_date}')

In [40]:
if __name__ == '__main__':

    current_date = datetime.date.today()

    spark = SparkSession.builder \
        .appName("generator_csv") \
        .config("spark.master", "local[*]") \
        .getOrCreate()

    schema = StructType([
        StructField('id', IntegerType(), False),
        StructField('name', StringType()),
        StructField('email', StringType()),
        StructField('city', StringType()),
        StructField('age', IntegerType()),
        StructField('salary', IntegerType()),
        StructField('registration_date', DateType())
    ])

    # вариант 1 ввода числа строк, дней и путь к директории с помощью input
    #    num_rows = int(input('Введите число генерируемых данных (количество строк): '))
    #    cnt_day = int(input('Введите количество дней, за которые нужно сгенерировать данные: '))
    #    output_path = input('Введите путь к директории для создания .csv файлов: ')  # '/content/sample_data'


    # вариант 2 с использованием библиотеки argparse, через аргумент
    # создаем парсер
    parser = argparse.ArgumentParser(description="Генерация синтетических данных.")
    # добавляем аргументы
    parser.add_argument('-r', '--rows', help='Число генерируемых данных (количество строк)',
                        nargs='?', type=int, default=100)
    parser.add_argument('-d', '--days', help='Число дней, за которые нужно сгенерировать данные',
                    nargs='?', type=int, default=5)
    parser.add_argument('--dir', help='путь к директории для сохранения .csv файлов',
                    nargs='?', type=str, default='/content/sample_data')

    # Разбор аргументов, игнорируя незнакомые(так как colab передает свои аргументы и будет ошибка)
    args, unknown = parser.parse_known_args()

    # Получаем значение из аргумента rows и days (если они был передан), либо используем дефолтное значение
    num_rows = args.rows
    cnt_day = args.days
    output_path = args.dir

    for days_ago in range(cnt_day):
        report_date = current_date - datetime.timedelta(days=days_ago)
        if report_date.day  % 2 != 0:
            data = generate_data(num_rows)
            df = create_df(spark, schema, data)
            columns_list = df.columns
            count_data = df.count()
            data_in_csv(df, report_date, output_path, count_data)
            #check_null_value(df, count_data, columns_list)


    spark.stop()

Было сгенерировано 100 строк данных, которые были записаны в файл '/content/sample_data/2025-02-21-dev.csv' за дату 2025-02-21
Было сгенерировано 100 строк данных, которые были записаны в файл '/content/sample_data/2025-02-19-dev.csv' за дату 2025-02-19


In [ ]:
############ весь код ниже

In [43]:
import random
import datetime
import os
import argparse
from transliterate import translit
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import col, when, count

# Список из 25 имен
prepared_list_names = [
    "Алексей", "Ольга", "Дмитрий", "Мария", "Иван",
    "Анна", "Сергей", "Елена", "Андрей", "Наталья",
    "Владимир", "Татьяна", "Михаил", "Светлана", "Кирилл",
    "Юлия", "Александр", "Екатерина", "Николай", "Ирина",
    "Василий", "Евгений", "Людмила", "Павел", "Роман"
]

# Список из 40 городов
prepared_list_cities = [
    "Москва", "Санкт-Петербург", "Новосибирск", "Екатеринбург", "Казань",
    "Нижний Новгород", "Челябинск", "Самара", "Омск", "Ростов-на-Дону",
    "Уфа", "Красноярск", "Воронеж", "Пермь", "Волгоград",
    "Краснодар", "Тольятти", "Ижевск", "Ульяновск", "Барнаул",
    "Тюмень", "Иркутск", "Саратов", "Хабаровск", "Ярославль",
    "Владивосток", "Махачкала", "Томск", "Оренбург", "Кемерово",
    "Новокузнецк", "Рязань", "Астрахань", "Пенза", "Липецк",
    "Тула", "Киров", "Брянск", "Чебоксары", "Калининград"
]

current_date = datetime.date.today()

# ф-ция генерации даты рождения
def generate_birth_date(age: int) -> datetime.date:
    # Расчет даты рождения для случайного возраста
    random_date_of_birth = current_date - datetime.timedelta(days=age * 365)

    return random_date_of_birth


# ф-ция генерации даты регистрации.
# Ограничения:
# регистрация возможна после достижения человеком пяти лет и до текущей даты.
def generate_registration_date(birth_date: datetime.date) -> datetime.date:
    # toordinal() переводит дату в дни, а fromordinal() обратно в дату
    start_date = birth_date.toordinal() + 5*365
    end_date = current_date.toordinal()
    random_registr_date = datetime.date \
        .fromordinal(random.randint(start_date, end_date))

    return random_registr_date

# ф-ции выбора случайного имени и города из отфильтрованного списка
def random_name() -> str:
  return random.choice([name for name in prepared_list_names if len(name) >= 5])

def random_city() -> str:
  return random.choice([city for city in prepared_list_cities if len(city) >= 7])

# ф-ция переименовывает .csv файлы в формат "текущий год-месяц-день-dev.csv" и
# удаляет лишние файлы
def rename_csv(output_path, date: datetime.date = current_date) -> str:
    files_list = os.listdir(output_path)
    new_filename = output_path + f'/{date}-dev.csv'

    for file in files_list:
        if file.startswith('part-') and file.endswith('.csv'):
            old_name = output_path + '/' + file
            os.rename(old_name, new_filename)
        elif not file.endswith('dev.csv'):
            os.remove(os.path.join(output_path, file))

    return new_filename

# ф-ция генерации данных
def generate_data(num_rows: int) -> list[tuple]:
    data = []

    # Рассчитываем количество NULL значений (не более 5% в каждом столбце)
    number_null_in_table = int(num_rows * 0.05)

    # Для каждого столбца создаем свой счетчик
    null_counts = {
        'name': 0,
        'email': 0,
        'city': 0,
        'age': 0,
        'salary': 0,
        'registration_date': 0
    }

    # Генерация данных
    for i in range(0, num_rows):
        name = random_name()
        city = random_city()
        age = random.randint(18, 95) # Генерация случайного возраста от 18 до 95 лет
        birth_date = generate_birth_date(age)
        registration_date = generate_registration_date(birth_date)
        #во время преобразования символов из ru в en translit заменяет "ь" на "'' (Ольга -> Ol'ga)
        # с помощью .replace("'", "") убираем "'" (Ольга -> Olga)
        email = translit(name.lower(), 'ru', reversed=True).replace("'", "")\
        + str(birth_date.year) + '_' + str(birth_date.day) + '@'\
        + random.choice(['ru', 'com'])
        salary = random.choice(range(100, 500)) * 10**3

        # промежуточная запись в словарь
        dict_data ={
            'name':name,
            'city':city,
            'age':age,
            'registration_date':registration_date,
            'email':email,
            'salary':salary
        }

        # Проверяем для каждого ключа (столбца) условие для вставки NULL значений
        for k, v in dict_data.items():
            #1. счетчик для столбца <= допустимому кол-ву NULL значений по столбцу
            #2. вероятность вставки NULL (если random.random() возвращает число меньше 0.05)
            if null_counts[k] <= number_null_in_table and random.random() < 0.05:
                dict_data[k] = None
                null_counts[k] += 1  # Увеличиваем счетчик для этого столбца


        # Создаем кортеж данных и добавляем его в итоговый список
        row_data = (i + 1, dict_data['name'], dict_data['email'], dict_data['city'],\
                    dict_data['age'], dict_data['salary'], dict_data['registration_date'])

        data.append(row_data)

    return data



# ф-ция создания 'DataFrame'
def create_df(spark: SparkSession, schema: StructType, data: list[tuple]) -> 'DataFrame':

    df = spark.createDataFrame(data=data, schema=schema)

    return df

# ф-ция проверки условия наличия NULL значений в данных
def check_null_value(df: 'DataFrame', count_data: int, columns_list: list) -> None:
    # ф-ция проверки условия на превышение NULL значений по столбцам (не прывышать 5% в каждом из столбцов)
    def check_null_percentage(column_name: str, cnt_null_colum: int, count_data: int) -> None:
        null_percentage = (cnt_null_colum / count_data) * 100
        if null_percentage > 5 and null_percentage <= 6:
            print(f'количество NULL в столбце "{column_name}" незначительно превышает 5% и равна {null_percentage:.2f}%')
        elif null_percentage > 6:
            print(f'количество NULL в столбце "{column_name}" ЗНАЧИТЕЛЬНО превышает 5% и равна {null_percentage:.2f}%')
        else:
            print(f'количество NULL в столбце "{column_name}" не превышает 5% и равна {null_percentage:.2f}%')

    for colum in columns_list:
        # количество NULL значений по столбцам
        cnt_null_colum = df.filter(col(colum).isNull()).count()
        check_null_percentage(colum, cnt_null_colum, count_data)


# ф-ция записи данных в .csv
def data_in_csv(df: 'DataFrame', report_date: datetime.date, output_path: str, count_data: int) -> None:

    df.coalesce(1).write.csv(output_path, header=True, mode="append")
    filename = rename_csv(output_path, report_date)


    print(f'Было сгенерировано {count_data} строк данных, \
которые были записаны в файл \'{filename}\' за дату {report_date}')

if __name__ == '__main__':

    spark = SparkSession.builder \
        .appName("generator_csv") \
        .config("spark.master", "local[*]") \
        .getOrCreate()

    schema = StructType([
        StructField('id', IntegerType(), False),
        StructField('name', StringType()),
        StructField('email', StringType()),
        StructField('city', StringType()),
        StructField('age', IntegerType()),
        StructField('salary', IntegerType()),
        StructField('registration_date', DateType())
    ])

    # вариант 1 ввода числа строк, дней и путь к директории с помощью input
    #    num_rows = int(input('Введите число генерируемых данных (количество строк): '))
    #    cnt_day = int(input('Введите количество дней, за которые нужно сгенерировать данные: '))
    #    output_path = input('Введите путь к директории для создания .csv файлов: ')  # '/content/sample_data'


    # вариант 2 с использованием библиотеки argparse, через аргумент
    # создаем парсер
    parser = argparse.ArgumentParser(description="Генерация синтетических данных.")
    # добавляем аргументы
    parser.add_argument('-r', '--rows', help='Число генерируемых данных (количество строк)',
                        nargs='?', type=int, default=100)
    parser.add_argument('-d', '--days', help='Число дней, за которые нужно сгенерировать данные',
                    nargs='?', type=int, default=5)
    parser.add_argument('--dir', help='путь к директории для сохранения .csv файлов',
                    nargs='?', type=str, default='/content/sample_data')

    # Разбор аргументов, игнорируя незнакомые(так как colab передает свои аргументы и будет ошибка)
    args, unknown = parser.parse_known_args()

    # Получаем значение из аргумента rows и days (если они был передан), либо используем дефолтное значение
    num_rows = args.rows
    cnt_day = args.days
    output_path = args.dir

    for days_ago in range(cnt_day):
        report_date = current_date - datetime.timedelta(days=days_ago)
        if report_date.day  % 2 != 0:
            data = generate_data(num_rows)
            df = create_df(spark, schema, data)
            columns_list = df.columns
            count_data = df.count()
            data_in_csv(df, report_date, output_path, count_data)
            #check_null_value(df, count_data, columns_list)


    spark.stop()

Было сгенерировано 100 строк данных, которые были записаны в файл '/content/sample_data/2025-02-21-dev.csv' за дату 2025-02-21
Было сгенерировано 100 строк данных, которые были записаны в файл '/content/sample_data/2025-02-19-dev.csv' за дату 2025-02-19
